#Compare data between properites SOW and SOW# #
Source: HubSpot

In [2]:
import pandas as pd
import sqlite3
import requests
import io

# Helper function to read CSV from Google Drive
def read_csv_from_gdrive(shared_id):
    url = f'https://drive.google.com/uc?id={shared_id}'
    response = requests.get(url)
    content = response.content.decode('utf-8')
    return pd.read_csv(io.StringIO(content))

# 1. Read the CSV file
shared_id = '1q8lkE5-_7n7YvZjDPqVCjs0sEyq_v0Zk'
df = read_csv_from_gdrive(shared_id)

# 2. Compare the "SOW ID" and "SOW #" columns
discrepancies = df[df['SOW ID'] != df['SOW #']]

# Extract the row numbers (index + 1) of discrepancies
discrepancies.loc[:, 'Row'] = discrepancies.index + 1

# Create a new DataFrame to store discrepancies and row numbers
discrepancy_df = discrepancies[['SOW ID', 'SOW #', 'Row']]

# 3. Create an SQLite database to store discrepancies
conn = sqlite3.connect('discrepancies.db')
discrepancy_df.to_sql('Discrepancies', conn, if_exists='replace', index=False)
conn.close()


<ipython-input-2-aad0b4c70270>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discrepancies.loc[:, 'Row'] = discrepancies.index + 1


In [3]:
discrepancy_df


,SOW ID,SOW #,Row
8,-420,2187-420,9
10,2186-418,-418,11
12,2160-416,-416,13
19,1783-410,-410,20
26,-403,2009-403,27
...,...,...,...
380,NaN,1005-103,381
381,396,1001-116,382
382,112,1012-112,383
383,NaN,1001-101,384


After examining the data, it was concluded that the `SOW#` property should be the primary source of truth due to its higher level of detail and more extensive information compared to `SOW`.

In [4]:
# Create a dataframe that only contains the data using SOW#
new_df = df[['Deal Name', 'SOW #']]

# Cleanup data from internal sheet
Source: [DealID sheet](https://docs.google.com/spreadsheets/d/19kCDF32F7Uh05Pue6ZnOUkBEGvwhQRJ2qozu97A3is8/edit#gid=1875314812)

In [5]:
# Clean up the data from the DealID Sheet

# 1. Read the CSV file
shared_id = '18ufpFp06NK4sn2_M2mXdDezwBgO1kHT1'
sheet_df = read_csv_from_gdrive(shared_id)

# 2. Remove the "TIME CREATED" and "LOOKUP" columns
sheet_df.drop(columns=['TIME CREATED', 'LOOKUP'], inplace=True)

# 3. Create "Expected SOW" column
def format_custid(x):
    if pd.isna(x):
        return 'NaN'
    elif x == int(x):
        return str(int(x))
    else:
        return str(x)

sheet_df['Expected SOW'] = sheet_df['CUSTID'].apply(format_custid) + '-' + sheet_df['DEALID'].astype(str)


# 4. Compare "Expected SOW" and "SOW ID"
discrepancies_new = sheet_df[sheet_df['Expected SOW'] != sheet_df['SOW ID']]

# 5. Extract the row numbers (index + 1) of discrepancies
discrepancies_new['Row'] = discrepancies_new.index + 1

# Create a new DataFrame to store discrepancies and row numbers
discrepancy_new_df = discrepancies_new[['Expected SOW', 'SOW ID', 'Row']]

sheet_df

<ipython-input-5-c9ed8d339bc8>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discrepancies_new['Row'] = discrepancies_new.index + 1


,DEAL NAME,CUSTID,DEALID,SOW ID,Expected SOW
0,Informatica MP VM Image,1000.0,100,1000-100,1000-100
1,"Google Extended Support (Beyond Accenture, etc)",1001.0,101,1001-101,1001-101
2,Integrated Billing/SaaS MP Dev,1000.0,102,1000-102,1000-102
3,MP Solution Dev for Fastly,1005.0,103,1005-103,1005-103
4,Vonage Data Science Dev,1006.0,104,1006-104,1006-104
...,...,...,...,...,...
895,NaN,NaN,995,NaN,NaN-995
896,NaN,NaN,996,NaN,NaN-996
897,NaN,NaN,997,NaN,NaN-997
898,NaN,NaN,998,NaN,NaN-998


In [6]:
discrepancy_new_df

,Expected SOW,SOW ID,Row
96,NaN-196,-196,97
113,NaN-213,-213,114
115,NaN-215,-215,116
119,NaN-219,-219,120
120,NaN-220,-220,121
...,...,...,...
895,NaN-995,NaN,896
896,NaN-996,NaN,897
897,NaN-997,NaN,898
898,NaN-998,NaN,899


# Compare data from Hubspot and Internal Sheet

In [7]:
print(sheet_df.columns)


Index(['DEAL NAME', 'CUSTID', 'DEALID', 'SOW ID', 'Expected SOW'], dtype='object')


In [10]:
# Custom sorting function
def custom_sort(sow_id):
    parts = str(sow_id).split('-')
    if len(parts) == 2 and len(parts[0]) == 4 and len(parts[1]) == 3:
        return int(parts[0]) * 1000 + int(parts[1])
    else:
        return float('inf')

# Reading and preprocessing the initial dataframe (You'll need to adjust the CSV reading for this one)
new_df = df[['Deal Name', 'SOW #']]

# Sorting new_df by SOW #
new_df_sorted = new_df.sort_values(by='SOW #', key=lambda x: x.apply(custom_sort))

# Reading and preprocessing the sheet_df (Adjust the read_csv function if not using Google Drive)
sheet_df = read_csv_from_gdrive('18ufpFp06NK4sn2_M2mXdDezwBgO1kHT1')


# Sorting sheet_df by SOW ID
sheet_df_sorted = sheet_df.sort_values(by='SOW ID', key=lambda x: x.apply(custom_sort))


# Combine sorted dataframes
combined_df_sorted = pd.concat([new_df_sorted.reset_index(drop=True), sheet_df_sorted.reset_index(drop=True)], axis=1)


In [11]:
combined_df_sorted

,Deal Name,SOW #,LOOKUP,DEAL NAME,CUSTID,DEALID,SOW ID,TIME CREATED
0,Informatica LLC: Informatica MP VM Image [1000...,1000-100,1,Informatica MP VM Image,1000.0,100,1000-100,NaN
1,Informatica LLC: Integrated Billing/SaaS MP De...,1000-102,1,Integrated Billing/SaaS MP Dev,1000.0,102,1000-102,NaN
2,Informatica: Master to Master replication [100...,1000-219,1,"Google Extended Support (Beyond Accenture, etc)",1001.0,101,1001-101,NaN
3,"Google: Extended Support (Beyond Accenture, et...",1001-101,1,WLMP Platform,1001.0,108,1001-108,NaN
4,Google: WLMP Platform [1001-108],1001-108,1,Google: TAM Renewal,1001.0,109,1001-109,NaN
...,...,...,...,...,...,...,...,...
895,NaN,NaN,0,NaN,NaN,604,NaN,NaN
896,NaN,NaN,0,NaN,NaN,605,NaN,NaN
897,NaN,NaN,0,NaN,NaN,606,NaN,NaN
898,NaN,NaN,0,NaN,NaN,608,NaN,NaN


## Inconsistent deals between Sheet and HubSpot

In [16]:
# Create lists of unique SOW values for each dataframe
new_df_sows = new_df['SOW #'].unique()
sheet_df_sows = sheet_df['SOW ID'].unique()

# Filter each dataframe to include only rows where the SOW is not in the other dataframe
new_df_unique_sows = new_df[new_df['SOW #'].isin(sheet_df_sows) == False].copy()
sheet_df_unique_sows = sheet_df[sheet_df['SOW ID'].isin(new_df_sows) == False].copy()

# Add a 'Source' column to each dataframe to indicate its origin
new_df_unique_sows['Source'] = 'HubSpot'
sheet_df_unique_sows['Source'] = 'Google Sheet'

# Concatenate the two filtered dataframes
combined_unique_sows = pd.concat([
    new_df_unique_sows.rename(columns={'SOW #': 'SOW'})[['Deal Name', 'SOW', 'Source']],
    sheet_df_unique_sows[['DEAL NAME', 'SOW ID', 'Source']].rename(columns={'DEAL NAME': 'Deal Name', 'SOW ID': 'SOW'})
], axis=0).reset_index(drop=True)


In [17]:
# Remove Duplicate Rows
combined_unique_sows.drop_duplicates(inplace=True)

# Handle Missing Values
# Drop rows with missing values
# combined_unique_sows.dropna(inplace=True)
# Or fill missing values with a default value (e.g., "Unknown")
combined_unique_sows.fillna("Unknown", inplace=True)

# Standardize Text Data
# Convert all text to uppercase
combined_unique_sows['Deal Name'] = combined_unique_sows['Deal Name'].str.upper()

# Trim White Spaces
combined_unique_sows['Deal Name'] = combined_unique_sows['Deal Name'].str.strip()
combined_unique_sows['SOW'] = combined_unique_sows['SOW'].str.strip()

# Remove Outliers or Invalid Data (example for SOW length not exceeding 10)
combined_unique_sows = combined_unique_sows[combined_unique_sows['SOW'].str.len() <= 10]

# Reorder Columns (Optional)
combined_unique_sows = combined_unique_sows[['Deal Name', 'SOW', 'Source']]

# Reset Index (Optional)
combined_unique_sows.reset_index(drop=True, inplace=True)

# Display cleaned dataframe
combined_unique_sows


,Deal Name,SOW,Source
0,LATAM AIRLINES: AML LATAM AIRLINES - TBD [2187...,2187-420,HubSpot
1,WORLDLINE: PUB TO GC [2121-419],-419,HubSpot
2,DUFRY GROUP: AML DUFRY GROUP - DYNAMIC PRICING...,-418,HubSpot
3,GRUPO SOMA: AML GRUPO SOMA - FORCASTING [2160-...,-416,HubSpot
4,CONCHA Y TORO: AML CONCHA Y TORO - ACCESS TO C...,2134-412,HubSpot
...,...,...,...
168,: AML GRUPO SOMA - NEW DEAL [-416],-416,Google Sheet
169,: AML DUFRY GROUP - NEW DEAL [-418],-418,Google Sheet
170,: PUB TO GC [-419],-419,Google Sheet
171,LATAM AIRLINES: AML LATAM AIRLINES - NEW DEAL ...,-420,Google Sheet


# Resolving the problem

In [19]:
# 1. Highlight Discrepancies

# Identify rows with discrepancies
df['Discrepancy'] = df['SOW ID'] != df['SOW #']

# 2. Suggest a Way to Fix

# If you want to actually fix the discrepancies in the original DataFrame:
df.loc[df['Discrepancy'], 'SOW ID'] = df.loc[df['Discrepancy'], 'SOW #']

# You can then drop the 'Discrepancy' column if you no longer need it
df.drop(columns='Discrepancy', inplace=True)

# Display the DataFrame to check
df


,Record ID,Deal Name,SOW ID,SOW #
0,15139838503,ConvergeOne: SUP for GC [1135-428],1135-428,1135-428
1,15139824649,Palo Alto Networks: GTM for GC [1007-427],1007-427,1007-427
2,15137749318,"cognaize: PUB to GC, AWS, AZURE [2258-426]",2258-426,2258-426
3,15139222023,A Harvey: AML A Harvey - Warehouse Slotting Op...,2371-425,2371-425
4,15139221912,Optiva: PUB to Azure [1143-424],1143-424,1143-424
...,...,...,...,...
380,9435439223,Fastly: MP Solution Dev for Fastly [1005-103],1005-103,1005-103
381,9435439216,Google: AI Tables Testing Phase 1 [1001-116],1001-116,1001-116
382,9468749674,Infoworks: Infoworks Cloud Migration [1012-112],1012-112,1012-112
383,9435592464,"Google: Extended Support (Beyond Accenture, et...",1001-101,1001-101


In [21]:
# Create a mask to filter rows where 'SOW' is not in the desired format
mask_invalid_format = ~df['SOW ID'].str.match(r'^\d{4}-\d{3}$')

# Filter rows and create a new DataFrame
invalid_sow_df = df[mask_invalid_format]


In [22]:
invalid_sow_df

,Record ID,Deal Name,SOW ID,SOW #
9,14653113122,Worldline: PUB To GC [2121-419],-419,-419
10,14653035832,Dufry Group: AML Dufry Group - Dynamic Pricing...,-418,-418
12,14544636501,Grupo Soma: AML Grupo Soma - Forcasting [2160-...,-416,-416
19,14519398776,Welspun USA: IQS Package [1571-410],-410,-410
101,9700949069,Resulticks Addendum: Publishing to GC Marketpl...,1183-345-A,1183-345-A
104,9691707022,MetaRouter: Publishing to GC Marketplace: [119...,1198-377-A,1198-377-A
114,9441019721,DuskRise: GCP Marketplace [1103-237-SUP],1103-237-SUP,1103-237-SUP
117,9367686197,Duskrise Addendum: Usage Based Dashboard [1103...,1103-237-A,1103-237-A
120,9335405126,Palo Alto: Technical Support Agreement [1007-2...,1007-241-SUP,1007-241-SUP
122,9274640733,ForgeRock: Technical Support Agreement [1057-3...,1057-378-SUP,1057-378-SUP
